In [1]:
# 카테고리1 - 페이지1 - 카테고리2 - 페이지1 ... 순차 : 카테고리 - 페이지 단위

import os
import requests
from bs4 import BeautifulSoup
from mysql.connector import connect, Error
from PIL import Image
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

# Declare as global variables
category_links = []
category_names = []
data = {}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
}

# 'output_links_all.txt' 파일에서 카테고리 링크 읽기
def read_category_links(filename):
    global category_links, category_names

    with open(filename, 'r') as file:
        lines = file.readlines()

    # 기존의 category_links와 category_names를 초기화
    category_links.clear()
    category_names.clear()

    for line in lines:
        # 첫 번째 하이픈에서만 분할
        first_hyphen_index = line.find('-')
        if first_hyphen_index != -1:
            category_link = line[:first_hyphen_index].strip()
            category_name = line[first_hyphen_index + 1:].strip()

            category_links.append(category_link)
            category_names.append(category_name)

    # 딕셔너리 초기화
    data.clear()
    for category_name in category_names:
        data[category_name] = {'articles': []}

# 'output_links_all.txt' 파일에서 카테고리 링크 읽기
read_category_links('output_links_all.txt')

# MariaDB 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'port': '3306',
    'raise_on_warnings': True,
}

# 'news_final' 데이터베이스 및 테이블 생성 함수
def create_news_table(cursor):
    # 'news02' 테이블이 이미 존재하는지 확인
    cursor.execute("SHOW TABLES LIKE 'news02'")
    result = cursor.fetchone()

    # 'news02' 테이블이 없으면 생성
    if not result:
        cursor.execute("""
            CREATE TABLE news02 (
                id INT AUTO_INCREMENT PRIMARY KEY,
                title VARCHAR(255) NOT NULL,
                content TEXT,
                link VARCHAR(255) NOT NULL,
                image_url VARCHAR(255),
                provided_time TIMESTAMP,
                category_id INT,
                FOREIGN KEY (category_id) REFERENCES categories(id)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_general_ci
        """)

def create_categories_table(cursor):
    # 'categories' 테이블이 이미 존재하는지 확인
    cursor.execute("SHOW TABLES LIKE 'categories'")
    result = cursor.fetchone()

    # 'categories' 테이블이 없으면 생성
    if not result:
        cursor.execute("""
            CREATE TABLE categories (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                link VARCHAR(255) NOT NULL
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_general_ci
        """)

def create_database(cursor):
    try:
        # 'news' 데이터베이스 확인
        cursor.execute("SHOW DATABASES LIKE 'news_final'")
        result = cursor.fetchone()

        # 'news' 데이터베이스 없으면 생성
        if not result:
            cursor.execute("CREATE DATABASE news_final")
            print("Database 'news_final' created.")

    except Error as create_db_err:
        print(f"Error creating or checking database: {create_db_err}")

    # 'news_final' 사용
    cursor.execute("USE news_final")

# 이미지 다운로드 및 저장 함수
def download_image(image_url):
    # 이미지 다운로드 및 저장
    if image_url and image_url != 'N/A':
        response = requests.get(image_url, headers=headers)
        if response.status_code == 200:
            # 이미지를 'images' 폴더에 저장
            valid_filename = re.sub(r'[\/:*?"<>|]', '', image_url.split("/")[-1])
            image_dir = "images"
            if not os.path.exists(image_dir):
                os.makedirs(image_dir)
            image_path = f"{image_dir}/{valid_filename}"
            with open(image_path, 'wb') as file:
                file.write(response.content)
            return image_path
    else:
        print(f"유효하지 않은 이미지 URL: {image_url}")
        return None

# 카테고리를 데이터베이스에 저장하는 함수
def save_category_to_database(cursor, category_name, category_link):
    # 카테고리가 이미 있는지 확인
    cursor.execute("SELECT id FROM categories WHERE name = %s", (category_name,))
    result = cursor.fetchone()

    # 카테고리가 없으면 삽입
    if not result:
        cursor.execute("INSERT INTO categories (name, link) VALUES (%s, %s)", (category_name, category_link))
        return cursor.lastrowid
    else:
        return result[0]

# provided_time 문자열을 timestamp로 변환하는 함수
def convert_to_timestamp(provided_time):
    if provided_time and provided_time != 'N/A':
        format_str = '%Y. %m. %d. %H:%M'
        dt = datetime.strptime(provided_time, format_str)
        return dt
    else:
        return 'N/A'

# 데이터를 데이터베이스에 저장하는 함수
def save_to_database(cursor, title, content, link, image_url, provided_time, category_name, category_link):
    cursor.execute("SELECT id FROM news02 WHERE link = %s", (link,))
    result = cursor.fetchone()

    if not result and content:  # content가 None이 아니고 비어 있지 않은 경우에만 저장
        insert_query = """
            INSERT INTO news02 (title, content, link, image_url, provided_time, category_id)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
        category_id = save_category_to_database(cursor, category_name, category_link)
        provided_time_timestamp = convert_to_timestamp(provided_time)
        insert_values = (title, content, link, image_url, provided_time_timestamp, category_id)
        cursor.execute(insert_query, insert_values)
        print('기사 저장 완료:', title)
        print('\n' + '-' * 50 + '\n')
    elif not content:
        print('기사 저장 실패 (content가 없음):', title)
        print('\n' + '-' * 50 + '\n')
    else:
        print('이미 저장된 기사입니다:', title)
        print('\n' + '-' * 50 + '\n')

# 다음 뉴스 기사의 내용을 크롤링하는 함수
def crawl_daum_news_content(article_soup):
    content_paragraphs = article_soup.find_all('p', {'dmcf-ptype': 'general'})
    content_text = '\n'.join([paragraph.get_text(strip=True) for paragraph in content_paragraphs])
    return content_text

#  다음 뉴스 기사를 페이지 단위로 크롤링
def crawl_daum_news_category_pages(category_link, category_name, start_page=1):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(options=chrome_options)

    try:
        selected_date = datetime.now()  # 현재 날짜 가져오기
        selected_date_str = selected_date.strftime("%Y%m%d")

        for page_number in range(start_page, start_page + 1):
            page_url = f"{category_link}?page={page_number}&regDate={selected_date_str}"

            # 디버깅 메시지 출력
            print(f"페이지에 접근 중: {page_url}")

            driver.get(page_url)

            if "다음뉴스" in driver.title:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                connection = connect(**db_config)
                cursor = connection.cursor()
                create_database(cursor)
                create_categories_table(cursor)
                create_news_table(cursor)
                save_category_to_database(cursor, category_name, category_link)

                article_links = soup.select('.tit_thumb > a.link_txt')

                for link in article_links:
                    article_url = link['href']
                    driver.get(article_url)
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.news_view')))

                    article_soup = BeautifulSoup(driver.page_source, 'html.parser')
                    title = article_soup.select_one('h3.tit_view')
                    title_text = title.get_text(strip=True) if title else 'N/A'
                    content_text = crawl_daum_news_content(article_soup)
                    image = article_soup.select_one('.news_view figure img')
                    image_url = image['src'] if image else 'N/A'
                    provided_time = article_soup.select_one('.num_date')
                    provided_time_text = provided_time.get_text(strip=True) if provided_time else 'N/A'
                    download_image(image_url)

                    # 데이터를 딕셔너리에 추가
                    data[category_name]['articles'].append({
                        'title': title_text,
                        'content': content_text,
                        'link': article_url,
                        'image_url': image_url,
                        'provided_time': provided_time_text,
                    })

                    # 데이터를 데이터베이스에 저장
                    save_to_database(cursor, title_text, content_text, article_url, image_url, provided_time_text, category_name, category_link)

                cursor.close()
                connection.commit()
                connection.close()
                print(f"MariaDB 연결 종료 - 페이지 {page_number}")

            else:
                print(f'페이지를 가져오지 못했습니다. 제목: {driver.title}')

    except Error as err:
        print(f"에러: {err}")

    finally:
        driver.quit()


# 크롤링 실행 
for i in range(len(category_links)):
    category_link = category_links[i]
    category_name = category_names[i]
    crawl_daum_news_category_pages(category_link, category_name)
    print(f"{category_name} 카테고리의 크롤링이 완료되었습니다.")


페이지에 접근 중: https://news.daum.net/breakingnews/?page=1&regDate=20240306
기사 저장 완료: “카카오, 먹통 사태 유감 표명하고 재발방지 노력하라”…법원 강제조정

--------------------------------------------------

기사 저장 완료: 근무자 격려하는 신원식 장관

--------------------------------------------------

유효하지 않은 이미지 URL: N/A
기사 저장 완료: 저축은행중앙회 "여권으로도 신분 증명"···알체라서 시스템 구축

--------------------------------------------------

기사 저장 완료: 신원식 장관, 공군항공우주의료원 방문

--------------------------------------------------

유효하지 않은 이미지 URL: N/A
기사 저장 완료: '한 방' 없었던 중국 경기부양책…총리 위상 축소도 주목

--------------------------------------------------

유효하지 않은 이미지 URL: N/A
기사 저장 완료: Time to bring in foreign caregivers

--------------------------------------------------

기사 저장 완료: [포토]정지석의 공격을 블로킹 하는 아르템(대한항공-우리카드)

--------------------------------------------------

유효하지 않은 이미지 URL: N/A
기사 저장 완료: [여기는 전남] “여순사건 역사왜곡·날조”…지역 반발 확산

--------------------------------------------------

유효하지 않은 이미지 URL: N/A
기사 저장 완료: [이슈대담] 위기의 ‘지역 대학’…새로운 해법 모색

--------------------------------